In [1]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 35.6 MB/s eta 0:00:00


In [2]:
!pip install numpy

In [3]:
!pip install scipy

In [16]:
import numpy as np
from scipy.sparse import csr_matrix
from pulp import LpVariable, LpProblem, LpMinimize, lpSum

# Create a dictionary of the knowledge graph
knowledge_graph = {
    ('A', 'B'): 2,
    ('B', 'C'): 3,
    ('C', 'A'): 1
}

# Create a list of the nodes in the knowledge graph
nodes = list(set([node for edge in knowledge_graph.keys() for node in edge]))

# Create a mapping between node names and indices
node_indices = {node: i for i, node in enumerate(nodes)}

# Create a matrix of the knowledge graph
matrix = csr_matrix((np.ones(len(knowledge_graph)), ([node_indices[edge[0]] for edge in knowledge_graph.keys()],
                                                     [node_indices[edge[1]] for edge in knowledge_graph.keys()])),
                    shape=(len(nodes), len(nodes)))

# Create a vector of the weights of the edges in the knowledge graph
weights = np.array([knowledge_graph[edge] for edge in knowledge_graph.keys()])

# Create a vector of the variables
variables = [LpVariable(node, lowBound=0) for node in nodes]

# Create the objective function
objective = lpSum([weights[i] * variables[node_indices[edge[0]]] for i, edge in enumerate(knowledge_graph.keys())])

# Create the constraints
constraints = []
constraints.append(lpSum([variables[i] for i in range(len(variables))]) == 1)
for i, edge in enumerate(knowledge_graph.keys()):
    constraints.append(lpSum(matrix[node_indices[edge[0]], j] * variables[j] for j in range(len(variables))) >= weights[i])

# Create the model
model = LpProblem("Linear Equation", LpMinimize)

# Add the objective function and constraints to the model
model += objective
for c in constraints:
    model += c

# Solve the model
model.solve()

# Print the solution
print("Optimal solution:")
for variable in variables:
    print(variable.name, ":", variable.varValue)

# Generate the linear equation
equation = ""
for i in range(len(variables)):
    if variables[i].varValue > 0:
        equation += str(weights[i] * variables[i].varValue) + variables[i].name + " + "
equation = equation[:-2] + " = 1"

# Print the linear equation
print("Linear equation:", equation)


Optimal solution:
A : 0.0
B : 2.0
C : 3.0
Linear equation: 6.0B + 3.0C  = 1


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
